In [ ]:
import hail as hl
from hail.plot import output_notebook, show

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
hl.init()
output_notebook()

#### Notebook to read in PVCF, filter to MAF > 1%, remove high missing sites and write out a bgen file

In [ ]:
recode_contigs = {f"{i}":f"chr{i}" for i in (list(range(1, 23)) + ["X", "Y"])}

In [ ]:
#read in file
path_2_file = "gs://pgr-wes-40k/GSA_chip_pvcf/CNCD_Freeze_Four.GT_hg38.pVCF.vcf.gz"

data = hl.import_vcf(path_2_file, reference_genome='GRCh38', force_bgz=True, \
                     contig_recoding=recode_contigs, array_elements_required=False, skip_invalid_loci=True)

In [ ]:
data.count()

In [ ]:
data.rows().show()

In [ ]:
#split multiallelic sites
bi_allelics = data.filter_rows(hl.len(data.alleles) == 2)
bi_allelics = bi_allelics.annotate_rows(a_index=1, was_split=False)
multi_allelics = data.filter_rows(hl.len(data.alleles) > 2)
multi_allelics_split = hl.split_multi_hts(multi_allelics)
data_split = multi_allelics_split.union_rows(bi_allelics)

In [ ]:
#filter to biallelic sites only
#missingness < 5%, aaf > 0.1%
#excess heterozygosity pval > 1E-16
data_split = hl.variant_qc(data_split)
data_split_filtered = data_split.filter_rows((data_split.variant_qc.call_rate > 0.97))
print(data_split.count())
data_split_filtered = data_split_filtered.filter_rows((data_split_filtered.variant_qc.AF[1] > 0.01) & \
                                                (data_split_filtered.variant_qc.AF[1] < 0.99))
print(data_split_filtered.count())
data_split_filtered = data_split_filtered.filter_rows(data_split_filtered.variant_qc.p_value_excess_het > 1E-16)
print(data_split_filtered.count())

In [ ]:
data_split_filtered = data_split_filtered\
.filter_rows(hl.is_snp(data_split_filtered.alleles[0], data_split_filtered.alleles[1]))
print(data_split_filtered.count())

In [ ]:
'''
p = hl.plot.histogram(hl.log10(data\
                               .variant_qc.p_value_excess_het))
show(p)
'''

In [ ]:
bi_allelics_filtered\
.write("pvcf_MAF1_qced_split.mt")

In [ ]:
mt = hl.read_matrix_table("/vcf_MAF1_qced_split.mt")
mt_autosomes = mt.filter_rows((mt.locus.contig != "chrX"))
mt_autosomes = mt_autosomes.filter_rows(mt_autosomes.locus.contig != "chrY")

In [ ]:
mt_autosomes.count()

In [ ]:
pruned_variant_table = hl.ld_prune(mt_autosomes.GT, r2=0.6, bp_window_size=75000, block_size=2048)
ld_pruned_mt = mt_autosomes.filter_rows(hl.is_defined(pruned_variant_table[mt_autosomes.row_key]))

In [ ]:
print(ld_pruned_mt.count())

In [ ]:
#Mean impute missing genotypes
ld_pruned_mt = ld_pruned_mt.annotate_rows(mean_gt = hl.agg.mean(ld_pruned_mt.GT.n_alt_alleles))
ld_pruned_mt = ld_pruned_mt.annotate_entries(n_alt_alleles=hl.coalesce(ld_pruned_mt.GT.n_alt_alleles, ld_prunded_mt.mean_gt))
                                             
#Add GP (genotype probability) field for BGEN file
ld_pruned_mt = ld_pruned_mt.annotate_entries(GP = GPs[ld_pruned_mt.GT.n_alt_alleles()])

In [ ]:
hl.export_bgen(ld_pruned_mt, 'pvcf_MAF1_qced_split_autosomes.bgen')